In [16]:
import sys, os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), ".."))) # lägg till training i path
from IPython.display import display, HTML
import pandas as pd
from src.data_loader import load_local_data, unload_local_data
bronze_data = load_local_data()
new_ads = bronze_data[['site', 'job_title', 'area', 'work_location','ingestion_ts', 'link']].sort_values(by='ingestion_ts', ascending=False)
new_ads["title_link"] = new_ads.apply(
        lambda x: f'<a href="{x["link"]}" target="_blank">{x["job_title"]}</a>', axis=1
    )

def return_styled_jobs_table(data): 
# Create a new column with job_title as a clickable link
    
 
    df_display = new_ads[["site", "title_link", "work_location", "ingestion_ts"]] # Pick only the required columns
    html_table = df_display.to_html(escape=False, index=False) # Convert to HTML and set text-align left only for the title_link column

    styled_html = f"""
    <style>
        td:nth-child(2), th:nth-child(2) {{
            text-align: left !important;
        }}
    </style>
    {html_table}
    """
    return HTML(styled_html)
styled_table = return_styled_jobs_table(new_ads)
display(styled_table)

site,title_link,work_location,ingestion_ts
Upgraded,Java-utvecklare,Stockholm/Karlstad,2025-09-16 16:16:55
Regent,Team på 2-4 penetrationstestare,Stockholm,2025-09-16 16:16:30
Ework,Expert System Engineer,Stockholm,2025-09-16 16:15:58
A Society,Arkitekt till Malmö,Malmö/Lund,2025-09-16 16:15:04
A Society,Senior Mechanical Engineer – Components and Products (Level 3),Ludvika,2025-09-16 16:15:04
Afry,Senior Engineer R&D - Codes and Standards,Norrbottens län,2025-09-16 16:13:48
Regent,Senior System Engineer – DevSecOps & Utvecklingsportaler,Stockholm,2025-09-16 15:40:50
Regent,En IT Support tekniker nivå 2- ASAP,Stockholm,2025-09-16 15:40:50
Ework,Test Engineer,Huskvarna,2025-09-16 15:40:44
Ework,Senior Platform Engineer,Stockholm,2025-09-16 15:40:44


In [22]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display, clear_output

# Kopia av job ads
df = new_ads.copy()

# Fördefinierade keywords
keywords_list = ["Analytiker", "Analys", "Bi", "Data", "ML", "Machine Learning", "Business Intelligence"]

# Skapa checkbox per keyword
checkboxes = [widgets.Checkbox(value=False, description=kw) for kw in keywords_list]

# Text-widget för eget keyword
custom_keyword = widgets.Text(
    value='',
    placeholder='Skriv eget keyword här',
    description='Eget keyword:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='200px')
)

# Output-area
output = widgets.Output()

# Funktion för filtrering
def filter_table(change=None):
    with output:
        clear_output()
        # Hämta valda checkbox keywords
        selected = [cb.description for cb in checkboxes if cb.value]
        # Lägg till text-widget keyword om det finns
        if custom_keyword.value.strip():
            selected.append(custom_keyword.value.strip())
        
        if not selected:
            table = df[["site", "job_title", "link", "ingestion_ts"]].copy()    
            display(table)
        else:
            mask = df['job_title'].str.contains('|'.join(selected), case=False, na=False)
            table = df[mask][["site", "job_title", "link", "ingestion_ts"]]
            display(table)

# Koppla onchange till varje checkbox och text-widget
for cb in checkboxes:
    cb.observe(filter_table, 'value')
custom_keyword.observe(filter_table, 'value')

# Visa checkboxar och text-widget horisontellt
controls = widgets.HBox(checkboxes + [custom_keyword])
display(controls, output)

# Visa hela tabellen initialt
filter_table()


Output()